## 고려대학교 구로병원 질병정보 크롤링

### 1. 주의사항
1. 총 46page로 구성되어있는데, 웹에서 크롤링을 중간에 자주 막습니다. (5page 단위로 할 때는 괜찮았음)
2. 함수로 모듈단위로 코드를 짜려했으나,일회성 코드라 그냥 바로 짰습니다.

In [9]:
import time
import os
import json
import csv
from datetime import datetime

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


# page 범위를 직접 조정하여 csv파일을 일일이 복붙했습니다
# (1 page ~ 47 page로 크롤링하면 중간에 서버가 끊깁니다
# write mode를 "a"가 아니라 "w"로 한 이유도, 중간에 서버가 끊기면 파일이 열리지 않아서 w로 작성한 후 일일이 수작업으로 복붙했습니다

In [36]:
kumc_cnt = open("kumc_cnt.csv", "w", encoding = 'utf-8-sig', newline='')
wr = csv.writer(kumc_cnt)
# csv file header 작성
wr.writerow(['질환명', '진료과', '질환 설명'])

# 1 ~ 46page까지 있습니다.
for page in range("페이지 입력하세요", "페이지 입력하세요"):
    url = 'http://guro.kumc.or.kr/info/diseaseSchList.do?cPage='+ str(page) +'&BOARD_ID=S001'
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    # page 접속 후 질병이름, 진료과, 질병이름 상세페이지에 대한 링크 획득
    for i in range(2, 12):
        try: 
            # dis_info (lifst)에 담은 후 바로 csv file에 작성합니다.
            dis_info = list()
            name = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(1)')[0].text.replace("\t", "")
            depart = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(2)')[0].text
            dis_info.append(name)
            dis_info.append(depart)
 
            # link를 따온 후 상세페이지로 들어갑니다.
            link = soup.select('tr:nth-of-type(2) > td.subject a')[0]['href']
            detail_url = 'http://guro.kumc.or.kr' + link
            n_req = requests.get(detail_url)
            n_cont = n_req.content
            n_soup = BeautifulSoup(n_cont, 'html.parser')

            # content에 상세 설명을 담습니다.
            content = n_soup.select('#contentArea > table')[0].text
            dis_info.append(content)

            #dis_info를 csv 파일에 적어줍니다.
            wr.writerow(dis_info)
        except:
            # 마지막 페이지에서 슬라이싱이 중간에 끊겨서 예외처리 추가함
            pass
        
kumc_cnt.close()       
    
    
    
    
    

45





냉방병(냉방 증후군)

 가정의학과



여름철 건강정보 Ⅱ

 가정의학과



여름철 건강정보 Ⅰ

 가정의학과



계절별 건강

 가정의학과



체중 감소

 가정의학과



헛구역질

 가정의학과



열사병

 가정의학과



여성부종

 가정의학과



비디오 디스플레이 터미널 증후군

 가정의학과



만성피로증후군

 가정의학과
46





딸꾹질

 가정의학과



독감

 가정의학과



비만

 가정의학과


# 연습장 (아래 내용은 보실 필요 없습니다)

In [ ]:
# 전체 page get 함수
def get_totalPage(url = "http://guro.kumc.or.kr/info/diseaseSchList.do?cPage=1&BOARD_ID=S001"):
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    total_num = soup.select("div.paging")[0]
    print(total_num)

In [138]:
# def 질병명, 진료과 수집기
#     (string) -> dictionary
#     Args:
#         url(string): 고려대 구로병원 건강정보 url
#     Return:
#         nameNdepart(dict):'질병명(key) - 진료과(value)'


def get_nameNdepart(url = "http://guro.kumc.or.kr/info/diseaseSchList.do?cPage=1&BOARD_ID=S001"):
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    nameNdepart = dict()
    for i in range(2, 12):
        name = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(1)')[0].text.split()[0]
        depart = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(2)')[0].text
        link = soup.select('tr:nth-of-type(2) > td.subject a')[0]['href']
        nameNdepart[name] = [depart, link]
        
    return nameNdepart
        
    

In [228]:
# def 질병명, 진료과 수집기
#     (string) -> dictionary
#     Args:
#         url(string): 고려대 구로병원 건강정보 url
#     Return:
#         nameNdepart(dict):'질병명(key) - 진료과(value)'


def get_nameNdepart(url = "http://guro.kumc.or.kr/info/diseaseSchList.do?cPage=1&BOARD_ID=S001"):
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    nameNdepart = dict()
    for i in range(2, 12):
        name = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(1)')[0].text.split()[0]
        depart = soup.select('tr:nth-of-type(' + str(i)+ ')> td:nth-of-type(2)')[0].text
        link = soup.select('tr:nth-of-type(2) > td.subject a')[0]['href']
        nameNdepart[name] = [[depart], [link]]
        
    return nameNdepart
        
    

In [258]:
def get_content(nameNdepart):
    """
    (dict) -> dict
    """
    nameNcnt = dict()
    
    for name in nameNdepart.keys():
        url = 'http://guro.kumc.or.kr' + nameNdepart[name][1][0]
        req = requests.get(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        content = soup.select('#contentArea > table')[0].text
        nameNcnt[name] = content 
     
    return nameNcnt

In [ ]:
if __name__ == "__main__":
    main()    

In [272]:
def main():
    ## 총 페이지수: 46page
    total_df = pd.DataFrame()
    for page in range(1, 5):
        print(page)
        url = 'http://guro.kumc.or.kr/info/diseaseSchList.do?cPage='+ str(page) +'&BOARD_ID=S001'
        nameNdepart = get_nameNdepart(url)
        nameNcontent = get_content(nameNdepart)
        
        df = pd.DataFrame(index = range(0,10), columns = ['질병 이름', '진료과', '내용'])
        
        for i, name in enumerate(nameNcontent):
            df['질병 이름'][i] = name
            df['진료과'][i] = nameNdepart[name][0]
            df['내용'][i] = nameNcontent[name]
        
        if page != 1:
            total_df = pd.concat([total_df, df], axis=1)
        else:
            pass
    total_df.to_csv("%s~total.csv" % (str(page)), encoding="utf-8")

In [273]:
if __name__ == "__main__":
    main()

1
2
3
4


In [234]:
df = pd.DataFrame(index = range(0,10), columns = ['질병 이름', '진료과', '내용'])

for i, name in enumerate(nameNcontent):
    df['질병 이름'][i] = name
    df['진료과'][i] = nameNdepart[name][0]
    df['내용'][i] = nameNcontent[name]


In [243]:
df2 = pd.DataFrame()

In [246]:
df5 = pd.concat([df2,df], axis=1)

In [247]:
df5

,질병 이름,진료과,내용
0,골반장기탈출증,산부인과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
1,뇌수막염,감염내과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
2,변실금,소화기내과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
3,지방간,간센터,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
4,수면무호흡증,이비인후·두경부외과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
5,협심증,심혈관센터(순환기),\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
6,베체트병,류마티스내과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
7,슬관절,정형외과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
8,무릎,정형외과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...
9,골관절염,정형외과,\n질병정보 상세 내용보기\n\n\n\n\n\n\n골반장기탈출증\n\n\n\n\n\...


In [184]:
a = get_content(get_nameNdepart())

In [253]:
b = get_nameNdepart()

In [190]:
a.keys()

dict_keys(['골반장기탈출증', '뇌수막염', '변실금', '지방간', '수면무호흡증', '협심증', '베체트병', '슬관절', '무릎', '골관절염'])

In [191]:
b.keys()

dict_keys(['골반장기탈출증', '뇌수막염', '변실금', '지방간', '수면무호흡증', '협심증', '베체트병', '슬관절', '무릎', '골관절염'])